In [ ]:
import kagglehub

# Download latest version
#path = kagglehub.dataset_download("manisha717/dataset-of-pdf-files")

/Users/seanhuang/opt/anaconda3/envs/cml/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 769M/769M [08:05<00:00, 1.66MB/s] 

Extracting files...


In [ ]:
print("Path to dataset files:", path)
#Path to dataset files: /Users/seanhuang/.cache/kagglehub/datasets/manisha717/dataset-of-pdf-files/versions/1


Path to dataset files: /Users/seanhuang/.cache/kagglehub/datasets/manisha717/dataset-of-pdf-files/versions/1


In [ ]:
#path = kagglehub.dataset_download("jenswalter/receipts")
print("Path to dataset files:", path)
# /Users/seanhuang/.cache/kagglehub/datasets/jenswalter/receipts/versions/15


100%|██████████| 176M/176M [02:12<00:00, 1.39MB/s] 

Extracting files...


Path to dataset files: /Users/seanhuang/.cache/kagglehub/datasets/jenswalter/receipts/versions/15


In [2]:
import fitz
import sys
import os
#extracting with partial ocr
#https://github.com/pymupdf/PyMuPDF-Utilities/blob/master/jupyter-notebooks/partial-ocr.ipynb
import pymupdf
import numpy 
import pytesseract
from pdfminer.pdfpage import PDFPage
from langchain_community.document_loaders import PyPDFLoader

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [12]:
from pathlib import Path
def extract_langchain( in_base : str, out_base: str, relative_file_path : str, ocr : bool = False): 
    in_path = os.path.abspath( in_base + relative_file_path)
    file_name = relative_file_path.replace(".pdf", ".txt")
    out_path = os.path.abspath(out_base +  file_name)
    print(in_path)
    print(out_path)
    loader = PyPDFLoader(in_path)
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    with open(out_path, "w", encoding="utf-8") as out:
        text_content = "\n\n".join([doc.page_content for doc in loader.load()])
        out.write(text_content)
        

def extract_normally( in_base : Path, out_base: Path, relative_file_path : Path, ocr : bool = False, resolve = False): 
    #in_path = os.path.abspath( in_base + relative_file_path)
    in_path = in_base / relative_file_path# Maintain folder structure
    print(in_path)
    # #extracted_file_path = extracted_base_folder / relative_path  # Maintain folder structure
    # stem = relative_file_path.stem + ("_ocr" if ocr else "_normal")  # Add suffix before extension
    # relative_file_name = relative_file_path.with_name(stem).with_suffix(".txt")  # Change extension
    # print("new relative:",relative_file_name)
    # if(resolve):
    #     out_path = (out_base / relative_file_name).resolve(strict=True)
    # else:
    #     out_path = out_base / relative_file_name  # Maintain folder structure
    # #print(out_path)
    # Extract original directory structure
    parent_dirs = relative_file_path.parent  # Preserve directory structure

    # Modify only the filename before changing the extension
    modified_filename = relative_file_path.stem + ("_ocr" if ocr else "_normal") + ".txt"

    # Reconstruct the full output path
    out_path = out_base / parent_dirs / modified_filename

    try: 

        doc = fitz.open(in_path)  # Open the PDF document
        # Ensure the output directory exists

        # Create output file name (replace .pdf with .txt)
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        with open(out_path, "w", encoding="utf-8") as out:
            for page in doc:  # Iterate through pages
                if not ocr:
                    text = page.get_text()  # Get page text
                else:
                    partial_tp = page.get_textpage_ocr(flags=0, full = True)
                    text = page.get_text(textpage=partial_tp)  # Get page text
                out.write(text)  # Write text to file
                out.write("\n" + ("-" * 50) + "\n")  # Page separator
        print(f"Extracted text saved to: {out_path}") 
    except Exception as e:
        tb = sys.exc_info()[2]
        lineno = tb.tb_lineno
        # Print the exception type, the error message, and the line number
        print(f"Exception Type: {type(e).__name__}")
        print(f"Error Message: {e}")
        print(f"Line Number: {lineno}")

def get_pdf_searchable_pages(fname):
    searchable_pages = []
    non_searchable_pages = []
    page_num = 0
    with open(fname, 'rb') as infile:

        for page in PDFPage.get_pages(infile):
            page_num += 1
            if 'Font' in page.resources.keys():
                searchable_pages.append(page_num)
            else:
                non_searchable_pages.append(page_num)
    if page_num > 0:
        if len(searchable_pages) == 0:
            print(f"Document '{fname}' has {page_num} page(s). "
                  f"Complete document is non-searchable")
        elif len(non_searchable_pages) == 0:
            print(f"Document '{fname}' has {page_num} page(s). "
                  f"Complete document is searchable")
        else:
            print(f"searchable_pages : {searchable_pages}")
            print(f"non_searchable_pages : {non_searchable_pages}")
    else:
        print(f"Not a valid document")
    
def needs_ocr(pdf_path):
    """Returns True if OCR is needed (no text detected), False otherwise."""
    doc = fitz.open(pdf_path)  # Open PDF
    for page in doc:
        text = page.get_text("text").strip()
        if text:  
            return False  # Found text → No OCR needed
    return True  # No text found → OCR needed



#### OCR produces better results

In [ ]:

#data start base folder:
base_folder = Path("/Users/seanhuang/Grad/adapter/Data/Raw")
extracted_base_folder = Path("/Users/seanhuang/Grad/adapter/Data/Extracted")
file_path = Path("Receipts/2018/ca/hotel/20181024_008.pdf")
text = extract_normally(base_folder,extracted_base_folder, file_path, True)
#text = extract_langchain(base_folder,extracted_base_folder, file_path, True)

file_path = Path("Receipts/2023/us/beerhouse_20231209_005.pdf")
text = extract_normally(base_folder,extracted_base_folder, file_path, True)
#text = extract_langchain(base_folder,extracted_base_folder, file_path, True)


Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Recipts/2018/ca/hotel/20181024_008_ocr.txt
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Recipts/2023/us/beerhouse_20231209_005_ocr.txt


#### Normal text fetching produces better results

In [121]:
#data start base folder:
base_folder = Path("/Users/seanhuang/Grad/adapter/Data/Raw")
extracted_base_folder = Path("/Users/seanhuang/Grad/adapter/Data/Extracted")
file_path = Path("WithFormat/Company Documents Dataset/CompanyDocuments/Inventory Report/monthly/monthly/StockReport_2016-07.pdf")
text = extract_normally(base_folder,extracted_base_folder, file_path ,False) 

file_path = Path("Pdf/0a29925ccc5e6299e132a73325956a3abef6dd26.pdf")
text = extract_normally(base_folder,extracted_base_folder, file_path ,False) 

file_path = Path("Pdf/2a85b52768ea5761b773be49b09d15f0b95415b0.pdf")
text = extract_normally(base_folder,extracted_base_folder, file_path , True) 


Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/WithFormat/Company Documents Dataset/CompanyDocuments/Inventory Report/monthly/monthly/StockReport_2016-07_normal.txt
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Pdf/0a29925ccc5e6299e132a73325956a3abef6dd26_normal.txt
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Pdf/2a85b52768ea5761b773be49b09d15f0b95415b0_ocr.txt


#### Create 2 versions of extracted text for LLM logical comparison

In [ ]:
#8Minutes
base_folder = Path("/Users/seanhuang/Grad/adapter/Data/Raw")
extracted_base_folder = Path("/Users/seanhuang/Grad/adapter/Data/Extracted")

for raw_file_path in base_folder.rglob("*.pdf"):  # Recursively get all files
    if raw_file_path.is_file() :  # Ignore directories
        relative_path = raw_file_path.relative_to(base_folder)
        #extracted_file_path = extracted_base_folder / relative_path  # Maintain folder structure
        print(f"Processing: {base_folder}")
        print(f"Relative: {relative_path}")
        print(f"Saving to: {extracted_base_folder}")
        extract_normally(base_folder,extracted_base_folder, relative_path, False)  # Extract text


Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: WithFormat/Company Documents Dataset/CompanyDocuments/invoices/invoice_10248.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/WithFormat/Company Documents Dataset/CompanyDocuments/invoices/invoice_10248_normal.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: WithFormat/Company Documents Dataset/CompanyDocuments/invoices/invoice_10274.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/WithFormat/Company Documents Dataset/CompanyDocuments/invoices/invoice_10274_normal.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: WithFormat/Company Documents Dataset/CompanyDocuments/invoices/invoice_10512.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/WithFormat/Company D

In [ ]:
#218minuntes
for raw_file_path in base_folder.rglob("*.pdf"):  # Recursively get all files
    if raw_file_path.is_file() :  # Ignore directories
        relative_path = raw_file_path.relative_to(base_folder)
        #extracted_file_path = extracted_base_folder / relative_path  # Maintain folder structure
        print(f"Processing: {base_folder}")
        print(f"Relative: {relative_path}")
        print(f"Saving to: {extracted_base_folder}")
        extract_normally(base_folder,extracted_base_folder, relative_path, True)  # Extract text

Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: WithFormat/Company Documents Dataset/CompanyDocuments/invoices/invoice_10248.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/WithFormat/Company Documents Dataset/CompanyDocuments/invoices/invoice_10248_ocr.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: WithFormat/Company Documents Dataset/CompanyDocuments/invoices/invoice_10274.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/WithFormat/Company Documents Dataset/CompanyDocuments/invoices/invoice_10274_ocr.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: WithFormat/Company Documents Dataset/CompanyDocuments/invoices/invoice_10512.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/WithFormat/Company Documen

Image too small to scale!! (1x36 vs min width of 3)
Line cannot be recognized!!
Image too small to scale!! (1x36 vs min width of 3)
Line cannot be recognized!!
Image too small to scale!! (2x36 vs min width of 3)
Line cannot be recognized!!
Image too small to scale!! (2x36 vs min width of 3)
Line cannot be recognized!!
Image too small to scale!! (1x36 vs min width of 3)
Line cannot be recognized!!
Image too small to scale!! (1x36 vs min width of 3)
Line cannot be recognized!!


Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Pdf/6HPMFPOTKN7J772QGZBHKGKYSNEYTF3I_ocr.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: Pdf/2544CYX3TC3T5QB2NTVXD3IUFM654GXK.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Pdf/2544CYX3TC3T5QB2NTVXD3IUFM654GXK_ocr.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: Pdf/ILRVVACIV2JDSO4LHLATCOCKSEQYZCMZ.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Pdf/ILRVVACIV2JDSO4LHLATCOCKSEQYZCMZ_ocr.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: Pdf/TLEMGWPUDGVX354P7HMCSBPERXLZWHHD.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Pdf/TLEMGWPUDGVX354P7HMCSBPERXLZWHHD_ocr.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: Pdf/B

Image too small to scale!! (2x36 vs min width of 3)
Line cannot be recognized!!
Image too small to scale!! (1x36 vs min width of 3)
Line cannot be recognized!!
Image too small to scale!! (2x36 vs min width of 3)
Line cannot be recognized!!
Image too small to scale!! (2x36 vs min width of 3)
Line cannot be recognized!!
Image too small to scale!! (2x36 vs min width of 3)
Line cannot be recognized!!
Image too small to scale!! (2x36 vs min width of 3)
Line cannot be recognized!!
Image too small to scale!! (2x36 vs min width of 3)
Line cannot be recognized!!
Image too small to scale!! (2x36 vs min width of 3)
Line cannot be recognized!!
Image too small to scale!! (1x36 vs min width of 3)
Line cannot be recognized!!


Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Pdf/WI6MQK44PH35QPVM66HY4CYIA7IW3XQO_ocr.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: Pdf/RUL7FW772XCJNPOZ34TKKQD3THLBCHH7.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Pdf/RUL7FW772XCJNPOZ34TKKQD3THLBCHH7_ocr.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: Pdf/PGJA7RZXFYTWRTYHWLSDEJFR2OKNYIP6.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Pdf/PGJA7RZXFYTWRTYHWLSDEJFR2OKNYIP6_ocr.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: Pdf/W5VHS6JXL2IZQKOADSVN6J2JWZPNWVC6.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Pdf/W5VHS6JXL2IZQKOADSVN6J2JWZPNWVC6_ocr.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: Pdf/Y

...

Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Recipts/2022/de/hit_20220607_002_ocr.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: Recipts/2022/de/hit_20220813_001.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Recipts/2022/de/hit_20220813_001_ocr.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: Recipts/2022/de/cloudfree_20221109_Invoice.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Recipts/2022/de/cloudfree_20221109_Invoice_ocr.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: Recipts/2022/de/dm_20221210_007.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Recipts/2022/de/dm_20221210_007_ocr.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: Recipts/2022/de

Image too small to scale!! (2x36 vs min width of 3)
Line cannot be recognized!!


Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Recipts/2022/de/hit-20220607_002_ocr.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: Recipts/2022/de/stadler-20220213.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Recipts/2022/de/stadler-20220213_ocr.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: Recipts/2022/de/hit-20220813_001.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Recipts/2022/de/hit-20220813_001_ocr.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: Recipts/2022/de/adidas_20220507_003.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Recipts/2022/de/adidas_20220507_003_ocr.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: Recipts/2022/de/fiveguys_20

Image too small to scale!! (2x36 vs min width of 3)
Line cannot be recognized!!


Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Recipts/2019/de/hotel/madison-502875_ocr.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: Recipts/2019/de/hotel/hampton_20190411.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Recipts/2019/de/hotel/hampton_20190411_ocr.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: Recipts/2019/de/hotel/hampton_24361.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Recipts/2019/de/hotel/hampton_24361_ocr.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw
Relative: Recipts/2019/de/hotel/madison_folio_g_cp_efolio5972171.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Recipts/2019/de/hotel/madison_folio_g_cp_efolio5972171_ocr.txt
Processing: /Users/seanhuang

#### Download and preprocess the paper linkes and summaries in 2018 Imagenetpapers dataset

In [8]:
# Define the base folder where you want to save the PDFs
base_folder = "/Users/seanhuang/Grad/adapter/Data/Raw/Reports"
file = "/Users/seanhuang/Grad/adapter/LLMAdapter/Preprocessing/Imagenetpapers.html"
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def extract_pdf_links_from(html_file):
    with open(html_file, "r", encoding="utf-8") as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, 'html.parser')
    links = soup.find_all('a', href=True)

    # Extract only links ending with .pdf
    pdf_links = [urljoin(html_file, link['href']) for link in links if link['href'].lower().endswith('.pdf')]
    
    return pdf_links

def download_pdfs(links, base_folder):

    os.makedirs(base_folder, exist_ok=True)  # Ensure base folder exists

    for pdf_link in links:
        try:
            pdf_filename = os.path.join(base_folder, os.path.basename(pdf_link))  # Get file name
            response = requests.get(pdf_link, stream=True)
            response.raise_for_status()  # Check for request errors

            with open(pdf_filename, 'wb') as f:
                for chunk in response.iter_content(1024):  # Download in chunks
                    f.write(chunk)
            print(f"Downloaded: {pdf_filename}")
        except Exception as e:
            print(f"Failed to download {pdf_link}: {e}")

In [128]:
links = extract_pdf_links_from(file)
download_pdfs(links, base_folder)

Downloaded: /Users/seanhuang/Grad/adapter/Data/Raw/Reports/2106.04803v2.pdf
Downloaded: /Users/seanhuang/Grad/adapter/Data/Raw/Reports/2106.04560v1.pdf
Downloaded: /Users/seanhuang/Grad/adapter/Data/Raw/Reports/2106.05974v1.pdf
Downloaded: /Users/seanhuang/Grad/adapter/Data/Raw/Reports/2003.10580v4.pdf
Downloaded: /Users/seanhuang/Grad/adapter/Data/Raw/Reports/2102.06171v1.pdf
Downloaded: /Users/seanhuang/Grad/adapter/Data/Raw/Reports/2106.11297v2.pdf
Downloaded: /Users/seanhuang/Grad/adapter/Data/Raw/Reports/2102.05918v2.pdf
Downloaded: /Users/seanhuang/Grad/adapter/Data/Raw/Reports/2010.01412v3.pdf
Downloaded: /Users/seanhuang/Grad/adapter/Data/Raw/Reports/2106.08254v1.pdf
Downloaded: /Users/seanhuang/Grad/adapter/Data/Raw/Reports/2003.08237v5.pdf
Downloaded: /Users/seanhuang/Grad/adapter/Data/Raw/Reports/2105.01601v4.pdf
Downloaded: /Users/seanhuang/Grad/adapter/Data/Raw/Reports/2010.11929v2.pdf
Downloaded: /Users/seanhuang/Grad/adapter/Data/Raw/Reports/2103.15808v1.pdf
Downloaded: 

In [14]:
base_folder = Path("/Users/seanhuang/Grad/adapter/Data/Raw/Reports")
extracted_base_folder = Path("/Users/seanhuang/Grad/adapter/Data/Extracted/Reports")

for raw_file_path in base_folder.rglob("*.pdf"):  # Recursively get all files
    if raw_file_path.is_file() :  # Ignore directories
        relative_path = raw_file_path.relative_to(base_folder)
        #extracted_file_path = extracted_base_folder / relative_path  # Maintain folder structure
        print(f"Processing: {raw_file_path}")
        print(f"Relative: {relative_path}")
        print(f"Saving to: {extracted_base_folder}")
        extract_normally(base_folder,extracted_base_folder, relative_path, False, resolve=True)  # Extract text

Processing: /Users/seanhuang/Grad/adapter/Data/Raw/Reports/2103.10697v2.pdf
Relative: 2103.10697v2.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted/Reports
/Users/seanhuang/Grad/adapter/Data/Raw/Reports/2103.10697v2.pdf
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Reports/2103.10697v2_normal.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw/Reports/2110.00476v1.pdf
Relative: 2110.00476v1.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted/Reports
/Users/seanhuang/Grad/adapter/Data/Raw/Reports/2110.00476v1.pdf
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Reports/2110.00476v1_normal.txt
Processing: /Users/seanhuang/Grad/adapter/Data/Raw/Reports/2103.16302v2.pdf
Relative: 2103.16302v2.pdf
Saving to: /Users/seanhuang/Grad/adapter/Data/Extracted/Reports
/Users/seanhuang/Grad/adapter/Data/Raw/Reports/2103.16302v2.pdf
Extracted text saved to: /Users/seanhuang/Grad/adapter/Data/Extracted/Reports/2103.16302v2_normal.txt
P

#### No need to OCR since the neurips papers are already in text format